<a href="https://colab.research.google.com/github/mitran27/GenerativeNetworks/blob/main/FineTuningLLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install datasets

!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install trl
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
from datasets import load_dataset
from peft import  LoraConfig, TaskType
import torch
from transformers import BitsAndBytesConfig
from transformers import LlamaTokenizerFast, TrainingArguments
from transformers import AutoTokenizer, AutoModelForCausalLM



In [4]:
dollyDataSet = load_dataset("databricks/databricks-dolly-15k")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
set(dollyDataSet["train"]["category"])

{'brainstorming',
 'classification',
 'closed_qa',
 'creative_writing',
 'general_qa',
 'information_extraction',
 'open_qa',
 'summarization'}

In [6]:
# drop sapmles greater than length 2200

reducedSamplesIndex = []
for idx, sample in enumerate(dollyDataSet["train"]):
  if(len(sample["instruction"])+len(sample["context"])<=1500):reducedSamplesIndex.append(idx)
len(reducedSamplesIndex)

13992

In [7]:
reducedSamples = dollyDataSet["train"].select(reducedSamplesIndex)
print(reducedSamples)
Dataset = reducedSamples.train_test_split(0.1)
print(Dataset)

Dataset({
    features: ['instruction', 'context', 'response', 'category'],
    num_rows: 13992
})
DatasetDict({
    train: Dataset({
        features: ['instruction', 'context', 'response', 'category'],
        num_rows: 12592
    })
    test: Dataset({
        features: ['instruction', 'context', 'response', 'category'],
        num_rows: 1400
    })
})


In [8]:
def createInstructionPrompt(record):
  return {"text":(
      f"below is an Instruction that describes a task."
      "Write a response that appropriately completes the request.\n\n"
      "### Instruction:\n"
      f"{record['instruction']}\n\n"
      f"### Response:\n"
      f"{record['response']}\n\n"
  )}
Dataset = Dataset.map(createInstructionPrompt)
print(Dataset,Dataset["train"][0])

Map:   0%|          | 0/12592 [00:00<?, ? examples/s]

Map:   0%|          | 0/1400 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'context', 'response', 'category', 'text'],
        num_rows: 12592
    })
    test: Dataset({
        features: ['instruction', 'context', 'response', 'category', 'text'],
        num_rows: 1400
    })
}) {'instruction': "What's the differences of the Wing Chun kongfu and the traditional Shaolin Kongfu?", 'context': '', 'response': "Wing chun was invented by a Buddhist nun who watched a snake and crane fighting. She adopted principles from both styles. She equated that to yin vs yang in terms of fighting style. \nIt's all about economy of movement, small quick strikes and close quarters combat. \nIt's meant to be effective for people of all ages and sizes and does not emphasize force. The crane stood there motionless and waded off the snakes quick strikes. It wasn't phased by the attacks. The posture and mindset of the crane and the quick strikes of the snake. \n\nShaolin was developed to ward off invaders. Usually e

In [9]:


qlora_config = LoraConfig(r=16, lora_alpha=12, lora_dropout = 0.05, bias="none", task_type="CAUSAL_LM")

bnb_config =BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_use_double_quant=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_compute_dtype=torch.bfloat16
)



In [10]:
def createPrompt(record):
  return (
      f"below is an Instruction that describes a task."
      "Write a response that appropriately completes the request.\n\n"
      f"{record['instruction']}\n\n"

  )

In [13]:
model = AutoModelForCausalLM.from_pretrained(
    "openlm-research/open_llama_7b",
    quantization_config = bnb_config
)


tokenizer = LlamaTokenizerFast.from_pretrained("openlm-research/open_llama_7b")
tokenizer.add_special_tokens({'pad_token':'[PAD]'})

prompt = createPrompt(Dataset["train"][145])
print(prompt)

inputs = tokenizer(prompt, return_tensors="pt", return_token_type_ids = False).to("cuda")
print(inputs)
outputs = model.generate(**inputs,max_new_tokens=100)

print(tokenizer.decode(outputs[0],skip_special_tokens=True))

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

below is an Instruction that describes a task.Write a response that appropriately completes the request.

What is Gyebangsan?


{'input_ids': tensor([[    1,  2519,   322,   363, 27681,   342,  8304,   260,  5149, 31843,
         18409,   260,  2924,   342, 19503, 28306,   266,  2980, 31843,    13,
            13,  5449,   322, 17251,  1369,   626, 22361, 31902,    13,    13]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1]], device='cuda:0')}
below is an Instruction that describes a task.Write a response that appropriately completes the request.

What is Gyebangsan?

Gyebangsan is a Korean word that means "a place where people gather to eat and drink."

What is the purpose of Gyebangsan?

The purpose of Gyebangsan is to provide a place where people can gather to eat and drink.

What is the history of Gyebangsan?

The history of Gyebangsan is that it was founded in 1990 by a group of

In [10]:
from trl import SFTTrainer

trainingArgs = TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        learning_rate=2e-4,
        max_steps=5000,
        output_dir="/content/sample_data",
        optim="paged_adamw_8bit",
        fp16=True
    )
supervised_finetuning_trainer = SFTTrainer(
    model,
    train_dataset=Dataset["train"],
    eval_dataset=Dataset["test"],
    tokenizer = tokenizer,
    dataset_text_field = "text",
    peft_config = qlora_config,
    max_seq_length = 1024,
    args= trainingArgs

)

supervised_finetuning_trainer.train()

Map:   0%|          | 0/12592 [00:00<?, ? examples/s]

Map:   0%|          | 0/1400 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:290: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(


In [14]:
# Load model directly
#Base Language Model (openlm-research/open_llama_v2_7b)
#Fine-Tuning Dataset (VMware/open-instruct)
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("VMware/open-llama-7b-v2-open-instruct")
model = AutoModelForCausalLM.from_pretrained("VMware/open-llama-7b-v2-open-instruct",    quantization_config = bnb_config)


inputs = tokenizer(prompt, return_tensors="pt", return_token_type_ids = False).to("cuda")
print(inputs)
outputs = model.generate(**inputs,max_new_tokens=100)

print(tokenizer.decode(outputs[0],skip_special_tokens=True))

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

{'input_ids': tensor([[    1,  2168,   325,   371, 25712,   347, 10801,   260,  4516, 29520,
          6925,   260,  2805,   347, 20488, 28963,   268,  2517, 29520,    13,
            13,  5616,   325, 19165,  1533, 28313,   278, 29584,    13,    13]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1]], device='cuda:0')}
below is an Instruction that describes a task.Write a response that appropriately completes the request.

What is Gyebangsan?

Gyebangsan is a Korean food that is made of fermented napa cabbage. It is typically eaten with rice and soup. It is a very popular side dish that is served at restaurants and cafes.
